# Checkers
I have decided to join the Board class and the Game class into one class. The way they have implemented the Board class makes it difficult to implement a checkers board. At the moment it just stores {coordinate, player}. In checkers we need to move players, delete players, promote players etc.. Of course we could modify it, but from an OOP point of view I think it is fine for the Checkers game to possess and control a board internally.

We start with two lists of pieces. The boolean next to the location (i,j) is the king status. If the piece is taken we replace its location with None. That is, the lists P[0] and P[1] will always have size 12.

In [20]:
# A standard game of checkers. 8x8 board.
class Checkers():
    def __init__(self):
        # initial positions
        W = [(x,y) for y in range(0,3) for x in range(y%2,8,2)]
        B = []# [(x,y) for y in range(5,8) for x in range(y%2,8,2)]
        B.append((3,3))
        B.append((3,5))
        B.append((1,5))
        B.append((1,3))
        B.append((7,3))
        self.P = {'W' : W, 'B' : B}
        # we record king status as a mapping from location to boolean
        W_King = {location : False for location in W}
        B_King = {location : False for location in B}
        self.Kings = {'W' : W_King,'B' : B_King}
        self.to_move = 'W'

    def remove_self_intersections(self, piece):
        # There are 4 possible moves.
        turn = self.to_move
        if(turn == 'W'):
            moves = []
            if(piece[1] < 7):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] + 1))
            if(self.Kings[turn][piece] and piece[1] > 0):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] - 1))
            available_moves = []
            for m in moves:
                if m in self.P['W']:
                    continue
                available_moves.append(m)
            return available_moves
            
        if(turn == 'B'):
            moves = []
            if(piece[1] > 0 ):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] - 1))
            if(self.Kings[turn][piece] and piece[1] < 0):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] + 1))
            available_moves = []
            for m in moves:
                if m in self.P['B']:
                    continue
                available_moves.append(m)
            return available_moves

    def capture_cycle(self, position, captured):
        print("in capture cycle")
        print("position: ", position)
        print("captured: ", captured)
        directions = []
        p = (position[0] - 1, position[1] - 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        p = (position[0] + 1, position[1] - 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        p = (position[0] - 1, position[1] + 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        p = (position[0] + 1, position[1] + 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        available_directions = []
        for d in directions:
            # we don't want to recapture and we can not do anything
            # if one of our pieces is  in the way. 
            if p not in captured and p not in self.P[self.to_move]:
                available_directions.append(d)
        for d in available_directions:
            rival = 'B' if self.to_move == 'W' else 'W'
            if d in self.P[rival]:
                # we have a rival piece, now we need to see if we can jump it.
                direction = (d[0] - position[0], d[1] - position[1])
                destination = (d[0] + direction[0], d[1] + direction[1])
                if destination[0] not in range(8) or destination[1] not in range(8): continue
                # We also need to check whether the destination is open. That is, clear.
                if destination in self.P['W'] or destination in self.P['B']: continue
                # At this point we have a valid square to land on.
                # We need to add the piece to captured and call capture_cycle again
                print("\t captured piece", d)
                print("\t destination", destination)
                captured.append(d)
                self.capture_cycle(destination, captured)
                
                
        
    
    def generate_action(self, i, piece, moves):
        captured = []
        other_player = 'B' if self.to_move == 'W' else 'W'
        other_player_piece_locations = self.P[other_player]
        for move in moves:
            if move in other_player_piece_locations:
                # We need to check whether the destination is valid
                direction = (move[0] - piece[0], move[1] - piece[1])
                destination = (move[0] + direction[0], move[1] + direction[1])
                if destination[0] not in range(8) or destination[1] not in range(8): continue
                # We also need to check whether the destination is open. That is, clear.
                if destination in self.P['W'] or destination in self.P['B']: continue
                # At this point we have a valid square to land on.
                print("entering capture cycle")
                print("destination: ", destination,"move: ", move)
                self.capture_cycle(destination, [move])
    
    def actions(self):
        actions = []
        for i in range(12):
            turn = self.to_move
            piece = self.P[turn][i]
            # If the piece is taken, just continue.
            if piece == None: continue
            moves = self.remove_self_intersections(piece)
            # If there are no moves ,just continue.
            if(len(moves) == 0): continue
            # action is a tuple:
            # (i, (l,m), [taken_pieces], make_king)
            print(i, piece, moves)
            action = self.generate_action(i, piece, moves)
            actions.append(action)
            # Loop through actions, if we find multiple actions that involve
            # taking at least a piece, we select the action that takes the most pieces. 
            

In [21]:
c = Checkers()
c.actions()

8 (0, 2) [(1, 3)]
entering capture cycle
destination:  (2, 4) move:  (1, 3)
in capture cycle
position:  (2, 4)
captured:  [(1, 3)]
	 captured piece (1, 5)
	 destination (0, 6)
in capture cycle
position:  (0, 6)
captured:  [(1, 3), (1, 5)]
	 captured piece (1, 5)
	 destination (2, 4)
in capture cycle
position:  (2, 4)
captured:  [(1, 3), (1, 5), (1, 5)]
	 captured piece (1, 5)
	 destination (0, 6)
in capture cycle
position:  (0, 6)
captured:  [(1, 3), (1, 5), (1, 5), (1, 5)]
	 captured piece (1, 5)
	 destination (2, 4)
in capture cycle
position:  (2, 4)
captured:  [(1, 3), (1, 5), (1, 5), (1, 5), (1, 5)]
	 captured piece (1, 5)
	 destination (0, 6)
in capture cycle
position:  (0, 6)
captured:  [(1, 3), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5)]
	 captured piece (1, 5)
	 destination (2, 4)
in capture cycle
position:  (2, 4)
captured:  [(1, 3), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5)]
	 captured piece (1, 5)
	 destination (0, 6)
in capture cycle
position:  (0, 6)
captured:  [(1, 3), (

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



	 captured piece (1, 5)
	 destination (0, 6)
in capture cycle
position:  (0, 6)
captured:  [(1, 3), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 

RecursionError: maximum recursion depth exceeded while calling a Python object